---
<h1 style="text-align: center;">IF867 - Introdução à Aprendizagem Profunda</h1>
<h2 style="text-align: center;">1ª atividade prática</h2>

---
---
*Discente:*

* Gabriel D'assumpção de Carvalho - gdc2@cin.ufpe.br

*Curso:*

* Ciências Atuariais - 7º Período

<div style="text-align: center;">
29/11/2024
</div>

# Instruções e Requisitos
- Objetivo: Implementar e treinar um Multilayer Perceptron (MLP), inteiramente em [NumPy](https://numpy.org/doc/stable/) ou [Numba](https://numba.readthedocs.io/en/stable/index.html), sem o uso de bibliotecas de aprendizado profundo.
- A atividade pode ser feita em dupla.

## Tarefas

__Implementação (50%):__

- Construa um MLP com uma camada de entrada, pelo menos duas camadas ocultas e uma camada de saída.
- Implemente pelo menos duas funções de ativação diferentes para as camadas ocultas; use Sigmoid e Linear para a camada de saída.
- Implemente forward e backpropagation.
- Implemente um otimizador de sua escolha, adequado ao problema abordado.
- Implemente as funções de treinamento e avaliação.

__Aplicação (30%):__

  Teste se os seus modelos estão funcionando bem com as seguintes tarefas:
  - Regressão
  - Classificação binária

__Experimentação (20%):__

  Teste os seus modelos com variações na arquitetura, no pré-processamento, etc. Escolha pelo menos uma das seguintes opções:
  - Variações na inicialização de pesos
  - Variações na arquitetura
  - Implementação de técnicas de regularização
  - Visualização das ativações e gradientes

***Bônus:*** Implemente o MLP utilizando uma biblioteca de machine learning (ex.: [PyTorch](https://pytorch.org/), [TensorFlow](https://www.tensorflow.org/?hl=pt-br), [tinygrad](https://docs.tinygrad.org/), [Jax](https://jax.readthedocs.io/en/latest/quickstart.html)) e teste-o em uma das aplicações e em um dos experimentos propostos. O bônus pode substituir um dos desafios de aplicação ou experimentos feitos em NumPy, ou simplesmente somar pontos para a pontuação geral.

## Datasets recomendados:
Aqui estão alguns datasets recomendados, mas fica a cargo do aluno escolher os datasets que utilizará na atividade, podendo escolher um dataset não listado abaixo.
- Classificação

  - [Iris](https://archive.ics.uci.edu/dataset/53/iris)
  - [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)
  - [CDC Diabetes Health Indicators](https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators)

- Regressão

  - [Air Quality](https://archive.ics.uci.edu/dataset/360/air+quality)
  - [Student Performance](https://archive.ics.uci.edu/dataset/320/student+performance)
  - [Wine Quality](https://archive.ics.uci.edu/dataset/186/wine+quality)

## Requisitos para Entrega

Um notebook Jupyter (de preferência, o link do colab) ou script Python contendo:

- Código: Implementação completa da MLP.
- Gráficos e Análises: Gráficos da curva de perda, ativações, gradientes e insights do treinamento, resultantes dos experimentos com parada antecipada e diferentes técnicas de regularização.
- Relatório: Um breve relatório detalhando o impacto de várias configurações de hiperparâmetros(ex.: inicialização de pesos, número de camadas ocultas e neurônios) e métodos de regularização no desempenho do modelo.


# Importações

## Bibliotecas

In [ ]:
import numpy as np # Data manipulation
import pandas as pd  # Data manipulation
import matplotlib.pyplot as plt # Data visualization

## Datasets

### Wine Quality

In [222]:
# Urls of the datasets
url_red = "https://raw.githubusercontent.com/gabrieldadcarvalho/deep_learning/refs/heads/master/exercise_list/01/data/wine/winequality-white.csv"
url_white = "https://raw.githubusercontent.com/gabrieldadcarvalho/deep_learning/refs/heads/master/exercise_list/01/data/wine/winequality-white.csv"

# Read of the arquives CSV
red = pd.read_csv(url_red, sep=";")
white = pd.read_csv(url_white, sep=";")

# Adding a column with the color of the wine
red["color"] = 1
white["color"] = 0

# Concatenation of the datasets
wine = pd.concat([red, white], axis=0)

# Print of the shape of the dataset
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1


## Iris Dataset

In [223]:
# Url of the iris dataset
url_iris = "https://raw.githubusercontent.com/gabrieldadcarvalho/deep_learning/refs/heads/master/exercise_list/01/data/iris/iris.data"

# Read of the arquives CSV
iris = pd.read_csv(url_iris, sep=",", header=None)

# Adding name of the variables
iris.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# 5 first rows
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Tratamento dos dados

Nessa sessão vai ser elaborado funções para fazer um breve tratamento no banco de dados para facilitar o aprendizado da rede.

In [224]:
# Function to print statistics
def print_stats(df, action, missing=False):
    if missing:
        print(f"Variables with missing values: \n{df.isna().sum()}")
        print("==" * 20)
        print(f"Total of missing values: \n{df.isna().sum().sum()}")
        print("==" * 20)
        if df.isna().any().any():
            print(f"Observations of missing values: \n{df[df.isna().any(axis=1)]}")
            print("==" * 20)
    else:
        print(f"Variables with duplicated values: \n{df.duplicated().sum()}")
        print("==" * 20)
        print(f"Total of duplicated values: \n{df.duplicated().sum()}")
        print("==" * 20)
        if df.duplicated().any():
            print(
                f"Observations of duplicated values: \n{df[df.duplicated(keep=False)]}"
            )
            print("==" * 20)

    print(f"Old shape: {df.shape}")

    return df


# Function for removing missing values
def drop_nan(df):
    # Show statistics
    print_stats(df, action="dropna", missing=True)

    # Drop of missing values
    df_cleaned = df.dropna()
    print(f"New shape: {df_cleaned.shape}")
    return df_cleaned


# Function for removing duplicated values
def drop_duplicates(df):
    # Show Statistics
    print_stats(df, action="drop_duplicates", missing=False)

    # Drop of duplicated values
    df_cleaned = df.drop_duplicates()
    print(f"New shape: {df_cleaned.shape}")
    return df_cleaned


def trans_categorical(df):
    print(df.info())
    print("==" * 20)
    categorical_cols = df.select_dtypes(include=["object"]).columns

    if len(categorical_cols) > 0:
        for column in categorical_cols:
            category_dict = {
                category: index
                for index, category in enumerate(pd.Categorical(df[column]).categories)
            }
            df[column] = df[column].map(category_dict)
        print(f"Column: {column} has {df[column].nunique()} categories")
        print("==" * 20)
        print("Transformation was made successfully!")
        print(category_dict)
        print("==" * 20)
        print("New info:")
        print(df.info())
    return df

## Wine

In [225]:
wine = trans_categorical(wine)

<class 'pandas.core.frame.DataFrame'>
Index: 9796 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         9796 non-null   float64
 1   volatile acidity      9796 non-null   float64
 2   citric acid           9796 non-null   float64
 3   residual sugar        9796 non-null   float64
 4   chlorides             9796 non-null   float64
 5   free sulfur dioxide   9796 non-null   float64
 6   total sulfur dioxide  9796 non-null   float64
 7   density               9796 non-null   float64
 8   pH                    9796 non-null   float64
 9   sulphates             9796 non-null   float64
 10  alcohol               9796 non-null   float64
 11  quality               9796 non-null   int64  
 12  color                 9796 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 1.0 MB
None


In [226]:
wine = drop_duplicates(wine)

Variables with duplicated values: 
1874
Total of duplicated values: 
1874
Observations of duplicated values: 
      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.0              0.27         0.36            20.7      0.045   
1               6.3              0.30         0.34             1.6      0.049   
2               8.1              0.28         0.40             6.9      0.050   
3               7.2              0.23         0.32             8.5      0.058   
4               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4851            6.4              0.33         0.44             8.9      0.055   
4855            7.1              0.23         0.39            13.7      0.058   
4856            7.1              0.23         0.39            13.7      0.058   
4879            6.6              0.34         0.40             8.1      0.046   

In [227]:
wine = drop_nan(wine)

Variables with missing values: 
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
color                   0
dtype: int64
Total of missing values: 
0
Old shape: (7922, 13)
New shape: (7922, 13)


Conforme observado nas funções anteriores, o dataframe do banco de dados de vinho não apresenta dados ausentes em nenhuma de suas variáveis. No entanto, foram identificados 1874 dados duplicados, sendo metade referente a vinhos de cor vermelha e a outra metade a vinhos de cor branca. Esse cenário ocorre devido ao banco de dados conter inicialmente 9796 registros, e após a remoção das duplicatas, o número de observações foi reduzido para 7922. Considerando esse ajuste, o impacto da remoção das duplicatas pode não ser tão significativo, uma vez que a base de dados continua representando uma quantidade substancial de informações.

Vale ressaltar que o objetivo desta atividade é desenvolver um algoritmo de MPL (Multilayer Perceptron) sem o uso de bibliotecas específicas de machine learning. Portanto, não nos aprofundamos em uma análise exploratória mais robusta. Não foi realizada uma investigação detalhada para avaliar o impacto das duplicatas na distribuição das variáveis ou para investigar se essas duplicações podem ser atribuídas a erros de arredondamento. Isso é especialmente relevante, pois das 12 variáveis independentes do conjunto de dados, 11 são contínuas, o que pode justificar variações numéricas pequenas que, eventualmente, resultam em duplicações.

## Iris

In [228]:
iris = trans_categorical(iris)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
None
Column: species has 3 categories
Transformation was made successfully!
{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
New info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null  

In [229]:
iris = drop_nan(iris)

Variables with missing values: 
sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64
Total of missing values: 
0
Old shape: (150, 5)
New shape: (150, 5)


In [230]:
iris = drop_duplicates(iris)

Variables with duplicated values: 
3
Total of duplicated values: 
3
Observations of duplicated values: 
     sepal_length  sepal_width  petal_length  petal_width  species
9             4.9          3.1           1.5          0.1        0
34            4.9          3.1           1.5          0.1        0
37            4.9          3.1           1.5          0.1        0
101           5.8          2.7           5.1          1.9        2
142           5.8          2.7           5.1          1.9        2
Old shape: (150, 5)
New shape: (147, 5)


No banco de dados Iris, nenhum dado ausente foi identificado nas variáveis. Foi realizada a remoção de 3 observações duplicadas, resultando em uma redução do número total de observações de 150 para 147. Essa remoção representou uma perda de apenas 2% da base de dados, o que pode ser considerado um impacto mínimo na integridade do conjunto de dados.